In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_distances

!ls "/content/drive/My Drive/Colab_Notebooks/data"

Mounted at /content/drive
 agnostic_bert_article_embeddings.pkl		 dbscan_Agnostic_clusters_analysis.csv
 agnostic_bert_first100_embeddings.pkl		 full_clusterd.pkl
 agnostic_bert_title_embeddings10k.pkl		 full_clustered.pkl
 agnostic_bert_title_embeddings.pkl		 full_data_fixed_dates.pkl
 all-the-news-2-1.csv				 full_dataset_Fine_embeddings.pkl
'anomalies_with_nearest_neighbors (1).gsheet'	 full_dataset_with_embeddings.pkl
'anomalies_with_nearest_neighbors2 (1).gsheet'	 full_data_with_dbscan.pkl
'anomalies_with_nearest_neighbors2 (2).gsheet'	 full_data_with_dbscan_revised.pkl
'anomalies_with_nearest_neighbors2 (3).gsheet'	 full_data_with_dbscan_title_fixed_eps.pkl
 anomalies_with_nearest_neighbors2.csv		'full_data_with_finetuned_embeddings&dbscan.pkl'
 anomalies_with_nearest_neighbors2.gsheet	 full_data_with_finetuned_embeddings.pkl
 anomalies_with_nearest_neighbors.csv		 Full_data_with_finetuned_embeddings.pkl
 anomalies_with_nearest_neighbors.gsheet	 full_data_with_finetuned_embedding

In [2]:
df =pd.read_pickle('/content/drive/My Drive/Colab_Notebooks/data/New_Normalized_Finetuned_DB.pkl')
df.columns
df.shape

(14994, 24)

In [3]:
# prompt: delete all columns and keep - date , titles, f100, ag_titles cluster, dbscan_titles,dbscan_f1000 clusters

cols_to_keep = ['date', 'title', 'F100', 'ag_title_cluster','ag_norm_bert_embedding', 'dbscan_titles','F100_finetuning_embedding_normalized','dbscan_F100_cluster']
df = df[cols_to_keep]
df.shape


(14994, 8)

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_distances


# מסנן את האאוטליירים
filtered_df = df[(df['dbscan_titles'] == -1) & (df['dbscan_F100_cluster'] == -1)]
filtered_df.shape

(9472, 8)

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_distances

##dbscan=-1 & f100 =-1
filtered_df = df[(df['dbscan_titles'] == -1) & (df['dbscan_F100_cluster'] == -1)]

# דוגם 50 שורות אקראיות
sampled_df = filtered_df.sample(n=50, random_state=42)

# פונקציה למציאת הנקודה הקרובה ביותר שלא כוללת את השורה עצמה
def find_nearest_point(embedding_col, reference_point, df, exclude_index):
    # מסיר את השורה הנוכחית מהחישוב
    df_filtered = df.drop(index=exclude_index)

    # ממיר את העמודה לרשימה של אמבדינגים
    embeddings = np.vstack(df_filtered[embedding_col].to_numpy())

    # ממיר את וקטור היעד לצורה תואמת
    reference_point = np.array(reference_point).reshape(1, -1)

    # מחשב מרחקים קוסיניים
    distances = cosine_distances(embeddings, reference_point)

    # מוצא את האינדקס של השורה הקרובה ביותר
    min_index = np.argmin(distances)
    min_distance = distances[min_index][0]

    # מחזיר את הכותרת, הכתבה המלאה, התאריך והמרחק
    nearest_row = df_filtered.iloc[min_index]

    return nearest_row['title'], nearest_row['F100'], nearest_row['date'], min_distance

# יצירת רשימות לאחסון תוצאות
nearest_ag_titles = []
nearest_ag_F100 = []
nearest_ag_dates = []
ag_distances = []
nearest_f100_titles = []
nearest_f100_F100 = []
nearest_f100_dates = []
f100_distances = []
ag_titles_clusters = []

# ריצה על 50 השורות שנדגמו
for index, row in sampled_df.iterrows():
    # מציאת הכתבה הקרובה ביותר לפי ag_norm_bert_embedding
    nearest_ag_title, nearest_ag_f100, nearest_ag_date, ag_dist = find_nearest_point(
        'ag_norm_bert_embedding', row['ag_norm_bert_embedding'], df, index
    )
    nearest_ag_titles.append(nearest_ag_title)
    nearest_ag_F100.append(nearest_ag_f100)
    nearest_ag_dates.append(nearest_ag_date)
    ag_distances.append(ag_dist)

    # מציאת הכתבה הקרובה ביותר לפי F100_finetuning_embedding_normalized
    nearest_f100_title, nearest_f100_f100, nearest_f100_date, f100_dist = find_nearest_point(
        'F100_finetuning_embedding_normalized', row['F100_finetuning_embedding_normalized'], df, index
    )
    nearest_f100_titles.append(nearest_f100_title)
    nearest_f100_F100.append(nearest_f100_f100)
    nearest_f100_dates.append(nearest_f100_date)
    f100_distances.append(f100_dist)

    # הוספת מספר הקלאסטר מה-DBSCAN
    ag_titles_clusters.append(row['ag_title_cluster'])

# הוספת התוצאות ל-DataFrame
sampled_df['nearest_ag_title'] = nearest_ag_titles
sampled_df['nearest_ag_F100'] = nearest_ag_F100
sampled_df['nearest_ag_date'] = nearest_ag_dates
sampled_df['ag_distance'] = ag_distances

sampled_df['nearest_f100_title'] = nearest_f100_titles
sampled_df['nearest_f100_F100'] = nearest_f100_F100
sampled_df['nearest_f100_date'] = nearest_f100_dates
sampled_df['f100_distance'] = f100_distances

sampled_df['ag_titles_cluster'] = ag_titles_clusters  # הוספת מספר הקלאסטר

# בחירת עמודות לשמירה
result_df = sampled_df[['title', 'F100', 'date',
                        'nearest_ag_title', 'nearest_ag_F100', 'nearest_ag_date', 'ag_distance',
                        'nearest_f100_title', 'nearest_f100_F100', 'nearest_f100_date', 'f100_distance',
                        'ag_titles_cluster']]

# שמירת התוצאה לקובץ CSV
result_df.to_csv('/content/drive/My Drive/Colab_Notebooks/data/anomalies_with_nearest_neighbors3.csv', index=False)

print("✅ קובץ CSV נוצר בהצלחה!")

✅ קובץ CSV נוצר בהצלחה!


In [9]:
import pandas as pd


# בוחרים 50 קלאסטרים ייחודיים
selected_clusters = df['dbscan_titles'].unique()[:50]

# מסנן את השורות כך שיכללו רק את 50 הקלאסטרים שנבחרו
filtered_df = df[df['dbscan_titles'].isin(selected_clusters)]

# בוחר דגימה אקראית מכל קלאסטר (לדוגמה 5 שורות לכל קלאסטר)
sampled_df = filtered_df.groupby('dbscan_titles').apply(lambda x: x.sample(n=min(len(x), 5), random_state=42)).reset_index(drop=True)

# שומר לקובץ CSV עם העמודות החשובות
sampled_df[['dbscan_titles', 'title', 'F100', 'date']].to_csv('/content/drive/My Drive/Colab_Notebooks/data/dbscan_50_clusters.csv', index=False)

print("✅ קובץ CSV עם 50 קלאסטרים נוצר בהצלחה!")

✅ קובץ CSV עם 50 קלאסטרים נוצר בהצלחה!


<ipython-input-9-7e9595d043b9>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = filtered_df.groupby('dbscan_titles').apply(lambda x: x.sample(n=min(len(x), 5), random_state=42)).reset_index(drop=True)
